# 🌙 Luna-LM Large — H100 Training

**Model:** Large (~350M params) — emb_dim=1024, n_heads=16, n_layers=24  
**GPU:** H100 (80GB VRAM)  
**Pipeline:** Pretrain → SFT

## Adımlar
1. GPU doğrula
2. Google Drive bağla
3. Repo kur
4. Corpus yükle
5. Pretrain
6. SFT
7. Modeli indir

In [ ]:
# 1. GPU Doğrula
import torch

print(f'GPU: {torch.cuda.get_device_name(0)}')
print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')
print(f'BF16 destekli: {torch.cuda.is_bf16_supported()}')

assert torch.cuda.is_available(), '❌ GPU bulunamadı! Runtime > Change runtime type > H100'
print('✅ GPU hazır!')

In [ ]:
# 2. Google Drive Bağla
from google.colab import drive
drive.mount('/content/drive')

# Checkpoint ve corpus Drive'a kaydedilecek
DRIVE_DIR = '/content/drive/MyDrive/LunaLM'
import os
os.makedirs(DRIVE_DIR, exist_ok=True)
print(f'✅ Drive bağlandı: {DRIVE_DIR}')

In [ ]:
# 3. Repo Kur
# Seçenek A: GitHub'dan clone et (repo public'se)
# !git clone https://github.com/iatagun/Luna-LM.git /content/Luna-LM

# Seçenek B: Drive'daki zip'i kullan
# !cp '/content/drive/MyDrive/LunaLM/Luna-LM.zip' /content/
# !unzip -q /content/Luna-LM.zip -d /content/

# Seçenek C: GitHub CLI ile clone (önerilen)
!git clone https://github.com/iatagun/Luna-LM.git /content/Luna-LM

import sys
sys.path.insert(0, '/content/Luna-LM')
os.chdir('/content/Luna-LM')
print('✅ Repo hazır!')

In [ ]:
# 4. Bağımlılıkları Yükle
!pip install -q transformers datasets

# Flash Attention (H100 için büyük hız kazanımı)
!pip install -q flash-attn --no-build-isolation

print('✅ Paketler yüklendi!')

In [ ]:
# 5. Corpus'u Yükle
# Seçenek A: Drive'dan kopyala
CORPUS_DRIVE = '/content/drive/MyDrive/LunaLM/foundation_corpus_clean.txt'
CORPUS_LOCAL = '/content/Luna-LM/foundation_corpus_clean.txt'

if os.path.exists(CORPUS_DRIVE):
    if not os.path.exists(CORPUS_LOCAL):
        print('Corpus Drive\'dan kopyalanıyor...')
        !cp '{CORPUS_DRIVE}' '{CORPUS_LOCAL}'
    size_gb = os.path.getsize(CORPUS_LOCAL) / 1024**3
    print(f'✅ Corpus hazır: {size_gb:.2f} GB')
else:
    print('⚠️  Corpus Drive\'da bulunamadı!')
    print('Lütfen foundation_corpus_clean.txt dosyasını şuraya yükleyin:')
    print(f'  {CORPUS_DRIVE}')
    print('Veya aşağıdaki hücreyi çalıştırarak doğrudan yükleyin:')

In [ ]:
# 5b. (Alternatif) Corpus'u doğrudan yükle
# Bu hücreyi sadece corpus Drive'da yoksa çalıştır

# from google.colab import files
# uploaded = files.upload()  # foundation_corpus_clean.txt seç
# !mv foundation_corpus_clean.txt /content/Luna-LM/

In [ ]:
# 6. PRETRAIN — Large Model (H100 Optimize)

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import json, random, time, os
from datetime import datetime

from luna.tokenizer import PretrainedTurkishTokenizer
from luna.data import create_dataloader_pretrained
from luna.model import GPTModel, MODEL_CONFIGS
from luna.generate import generate_text

# ==========================================
# AYARLAR — H100 (80GB) için optimize
# ==========================================
MODEL_SIZE    = 'large'   # emb_dim=1024, n_heads=16, n_layers=24
BATCH_SIZE    = 32        # H100: 80GB → büyük batch
CONTEXT_LEN   = 1024      # Large model için daha uzun context
NUM_EPOCHS    = 3
LEARNING_RATE = 6e-4      # Large model için biraz daha yüksek
WEIGHT_DECAY  = 0.1
EVAL_FREQ     = 500
EVAL_ITER     = 20
MAX_LINES     = None      # Tüm corpus (3.6GB)
USE_BF16      = True      # H100 BF16 native destekler — çok daha hızlı

device = torch.device('cuda')
dtype  = torch.bfloat16 if USE_BF16 else torch.float32

print(f'Device: {device}')
print(f'DType:  {dtype}')
print(f'Model:  {MODEL_SIZE}')
print(f'Batch:  {BATCH_SIZE}')
print(f'Ctx:    {CONTEXT_LEN}')

In [ ]:
# Tokenizer
tokenizer = PretrainedTurkishTokenizer('dbmdz/bert-base-turkish-cased')
vocab_size = tokenizer.vocab_size
print(f'Vocab: {vocab_size:,}')

In [ ]:
# Corpus Yükle
corpus_path = '/content/Luna-LM/foundation_corpus_clean.txt'

print('Corpus yükleniyor...')
lines = []
with open(corpus_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if MAX_LINES and i >= MAX_LINES:
            break
        line = line.strip()
        if line:
            lines.append(line)
        if (i+1) % 500000 == 0:
            print(f'  {i+1:,} satır okundu...')

print(f'✅ {len(lines):,} satır yüklendi')

random.seed(42)
random.shuffle(lines)

split = int(0.95 * len(lines))   # 95/5 split — büyük corpus
train_text = '\n'.join(lines[:split])
val_text   = '\n'.join(lines[split:])
del lines

print(f'Train: {len(train_text)/1e9:.2f}B karakter')
print(f'Val:   {len(val_text)/1e6:.1f}M karakter')

In [ ]:
# DataLoader
train_loader = create_dataloader_pretrained(
    train_text, tokenizer,
    batch_size=BATCH_SIZE, max_length=CONTEXT_LEN, stride=CONTEXT_LEN, shuffle=True
)
val_loader = create_dataloader_pretrained(
    val_text, tokenizer,
    batch_size=BATCH_SIZE, max_length=CONTEXT_LEN, stride=CONTEXT_LEN, shuffle=False
)

print(f'Train batches: {len(train_loader):,}')
print(f'Val batches:   {len(val_loader):,}')

In [ ]:
# Model Oluştur
cfg = MODEL_CONFIGS[MODEL_SIZE]
model_config = {
    'vocab_size':      vocab_size,
    'context_length':  CONTEXT_LEN,
    'emb_dim':         cfg['emb_dim'],
    'n_heads':         cfg['n_heads'],
    'n_layers':        cfg['n_layers'],
    'drop_rate':       0.1,
    'qkv_bias':        False,
}

model = GPTModel(model_config)

# BF16 için cast
if USE_BF16:
    model = model.to(dtype=torch.bfloat16)

model = model.to(device)

# torch.compile — H100'de ~%30 hız artışı
model = torch.compile(model)

total_params = sum(p.numel() for p in model.parameters())
print(f'✅ Model hazır: {total_params/1e6:.1f}M parametre')
print(f'   emb_dim={cfg["emb_dim"]}, n_heads={cfg["n_heads"]}, n_layers={cfg["n_layers"]}')

In [ ]:
# Optimizer & Scheduler
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    betas=(0.9, 0.95),   # GPT-3 style
    eps=1e-8,
    fused=True           # H100 için fused optimizer
)

total_steps   = len(train_loader) * NUM_EPOCHS
warmup_steps  = int(total_steps * 0.02)   # %2 warmup — büyük model

def lr_lambda(step):
    if step < warmup_steps:
        return step / max(warmup_steps, 1)
    progress = (step - warmup_steps) / max(total_steps - warmup_steps, 1)
    import math
    return 0.1 + 0.9 * 0.5 * (1.0 + math.cos(math.pi * progress))  # min_lr = 10% of max

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

print(f'Total steps:  {total_steps:,}')
print(f'Warmup steps: {warmup_steps:,}')

In [ ]:
# Save Dir (Drive'a kaydet — session bitse bile korunur)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir  = f'{DRIVE_DIR}/pretrain_large_{timestamp}'
os.makedirs(save_dir, exist_ok=True)

with open(f'{save_dir}/config.json', 'w') as f:
    json.dump({
        'model_config': model_config,
        'training_config': {
            'model_size': MODEL_SIZE,
            'batch_size': BATCH_SIZE,
            'num_epochs': NUM_EPOCHS,
            'learning_rate': LEARNING_RATE,
            'context_length': CONTEXT_LEN,
            'use_bf16': USE_BF16,
        },
        'tokenizer': 'dbmdz/bert-base-turkish-cased',
        'timestamp': timestamp,
    }, f, indent=2)

print(f'✅ Save dir: {save_dir}')

In [ ]:
# PRETRAIN LOOP
from torch.cuda.amp import autocast

train_losses, val_losses, tokens_seen_log = [], [], []
tokens_seen  = 0
global_step  = 0
best_val_loss = float('inf')

print('='*60)
print('PRETRAIN BAŞLIYOR')
print('='*60)

for epoch in range(NUM_EPOCHS):
    model.train()
    ep_start = time.time()
    
    for input_batch, target_batch in train_loader:
        input_batch  = input_batch.to(device)
        target_batch = target_batch.to(device)
        
        optimizer.zero_grad()
        
        with autocast(dtype=torch.bfloat16, enabled=USE_BF16):
            logits = model(input_batch)
            loss = torch.nn.functional.cross_entropy(
                logits.flatten(0, 1), target_batch.flatten()
            )
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        tokens_seen += input_batch.numel()
        global_step += 1
        
        if global_step % EVAL_FREQ == 0:
            model.eval()
            with torch.no_grad():
                # Train loss sample
                tl = 0
                for k, (ib, tb) in enumerate(train_loader):
                    if k >= EVAL_ITER: break
                    with autocast(dtype=torch.bfloat16, enabled=USE_BF16):
                        lg = model(ib.to(device))
                        tl += torch.nn.functional.cross_entropy(lg.flatten(0,1), tb.to(device).flatten()).item()
                tl /= EVAL_ITER
                
                # Val loss sample
                vl = 0
                for k, (ib, tb) in enumerate(val_loader):
                    if k >= EVAL_ITER: break
                    with autocast(dtype=torch.bfloat16, enabled=USE_BF16):
                        lg = model(ib.to(device))
                        vl += torch.nn.functional.cross_entropy(lg.flatten(0,1), tb.to(device).flatten()).item()
                vl /= EVAL_ITER
            
            train_losses.append(tl)
            val_losses.append(vl)
            tokens_seen_log.append(tokens_seen)
            
            lr = optimizer.param_groups[0]['lr']
            print(f'Ep {epoch+1} | Step {global_step:,} | '
                  f'Train: {tl:.4f} | Val: {vl:.4f} | '
                  f'LR: {lr:.2e} | Tokens: {tokens_seen/1e6:.1f}M')
            
            if vl < best_val_loss:
                best_val_loss = vl
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'epoch': epoch, 'step': global_step,
                    'val_loss': vl, 'tokens_seen': tokens_seen,
                }, f'{save_dir}/best_model.pt')
                print(f'  ✅ Best model! Val: {vl:.4f}')
            
            model.train()
    
    ep_time = (time.time() - ep_start) / 60
    print(f'\nEpoch {epoch+1} tamamlandı ({ep_time:.1f} dk)\n')
    
    # Epoch checkpoint
    torch.save({'model_state_dict': model.state_dict(), 'epoch': epoch},
               f'{save_dir}/epoch_{epoch+1}.pt')

print('✅ PRETRAIN TAMAMLANDI!')
print(f'Best val loss: {best_val_loss:.4f}')

In [ ]:
# Loss Grafiği
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(tokens_seen_log, train_losses, label='Train')
ax.plot(tokens_seen_log, val_losses,   label='Val', linestyle='--')
ax.set_xlabel('Tokens Seen')
ax.set_ylabel('Loss')
ax.set_title('Luna-LM Large — Pretrain')
ax.legend(); ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(f'{save_dir}/pretrain_loss.png', dpi=200)
plt.show()

---
## SFT — Alpaca Turkish (82K)

In [ ]:
# SFT Dataseti İndir
from datasets import load_dataset
import json

SFT_JSONL = '/content/Luna-LM/sft/sft_dataset.jsonl'

if not os.path.exists(SFT_JSONL):
    print('SFT dataset indiriliyor...')
    ds = load_dataset('cenfis/alpaca-turkish-combined')
    with open(SFT_JSONL, 'w', encoding='utf-8') as f:
        for row in ds['train']:
            user = row['instruction']
            if row.get('input'):
                user += f"\n{row['input']}"
            if user and row['output']:
                f.write(json.dumps({'user': user, 'assistant': row['output']}, ensure_ascii=False) + '\n')
    print(f'✅ SFT dataset hazır: {SFT_JSONL}')
else:
    print(f'✅ SFT dataset mevcut: {SFT_JSONL}')

In [ ]:
# SFT Eğitimi
import subprocess
result = subprocess.run(
    ['python', 'sft/train_sft.py'],
    capture_output=False,
    text=True,
    cwd='/content/Luna-LM'
)

# NOT: train_sft.py'deki BATCH_SIZE'ı H100 için 32 yap
# ve en son pretrain checkpoint'ini doğru göster

In [ ]:
# Modeli Drive'a Kopyala (session bitince korunur)
!cp -r /content/Luna-LM/checkpoints/sft_* '{DRIVE_DIR}/'
print('✅ SFT model Drive\'a kopyalandı!')

# zip'le ve indir (isteğe bağlı)
# !zip -r /content/luna_large_sft.zip '{save_dir}/best_model.pt'
# from google.colab import files
# files.download('/content/luna_large_sft.zip')

In [ ]:
# Hızlı Test
from luna.utils import load_model
from luna.generate import generate_text

SYSTEM_PROMPT = ('Senin adın Luna. Amacın insanlara yardımcı olmak ve sorulara açık, '
                 'anlaşılır cevaplar vermektir. Emin olmadığın konularda bunu belirtir, '
                 'uydurma bilgi eklemezsin.')

def chat(model, tokenizer, device, question, **kwargs):
    prompt = f'<system>{SYSTEM_PROMPT}</system>\n<user>{question}</user>\n<assistant>'
    out = generate_text(model, tokenizer, device, prompt, max_new_tokens=200,
                        temperature=0.7, top_k=50, repetition_penalty=1.2, **kwargs)
    if '<assistant>' in out:
        ans = out.split('<assistant>')[-1]
        for s in ['</assistant>', '<user>', '<system>', '[SEP]']:
            ans = ans.split(s)[0]
        return ans.strip()
    return out

# En son SFT checkpoint'i yükle
import glob
sft_dirs = sorted(glob.glob('/content/Luna-LM/checkpoints/sft_*'))
model_t, tokenizer_t, _ = load_model(sft_dirs[-1], device)

test_questions = [
    'Güneş hangi yönden doğar?',
    'Yapay zeka nedir?',
    'Türkiye\'nin başkenti neresidir?',
]

for q in test_questions:
    print(f'❓ {q}')
    print(f'🤖 {chat(model_t, tokenizer_t, device, q)}\n')